In [ ]:
# Git Clone
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 2.39 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Linux and Windows are supported, but we recommend Linux for performance and compatibility reasons.
* 1–8 high-end NVIDIA GPUs with at least 12 GB of memory. We have done all testing and development using NVIDIA DGX-1 with 8 Tesla V100 GPUs.
* 64-bit Python 3.7 and PyTorch 1.7.1. See https://pytorch.org/ for PyTorch install instructions.
* CUDA toolkit 11.0 or later. Use at least version 11.1 if running on RTX 3090. (Why is a separate CUDA toolkit installation required? See comments in #2.)
* Python libraries: pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3. We use the Anaconda3 2020.11 distribution which installs most of these by default.

[ciekawe](https://oscar-guarnizo.medium.com/review-image2stylegan-embedding-an-image-into-stylegan-c7989e345271)


In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.3 MB/s eta 0:04:02tcmalloc: large alloc 1147494400 bytes == 0x39d8e000 @  0x7f4056acd615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.3 MB/s eta 0:01:19tcmalloc: large alloc 1434370048 bytes == 0x7e3e4000 @  0x7f4056acd615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 32.2 MB/s 
     |████████████████████████████████| 108 kB 74.0 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 


#Run StyleGAN2 From Python Code
Add the StyleGAN folder to Python so that you can import it. The code below is based on code from NVidia. This actually generates your images.

In [ ]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [ ]:
!gdown https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Downloading...
From: https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
To: /content/ffhq.pkl
100% 382M/382M [00:02<00:00, 185MB/s]


In [ ]:
import torch
import pickle


with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
G.z_dim

512

In [ ]:
img = G(z, c)   

In [ ]:
img.shape

torch.Size([1, 3, 1024, 1024])

In [ ]:
img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'z_vexctor_image.png')

In [ ]:
w = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)

In [ ]:
w.shape

torch.Size([1, 18, 512])

In [ ]:
w = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)
img = G.synthesis(w, noise_mode='const', force_fp32=True)

In [ ]:
img.shape

torch.Size([1, 3, 1024, 1024])

In [ ]:
import numpy as np
import PIL

img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'Wodeczka.png')

# Sprawdzic

czy 18,512 to poprostu 18* 1,512 (ten sam wektor bo zapewne tak)

Assumption:
w = z * 18 


# Encoder or projector

Image 2 Stylegan [PPL Loss i inne](https://arxiv.org/pdf/1904.03189.pdf)

In [ ]:
!wget https://download.pytorch.org/models/vgg16-397923af.pth

--2022-06-03 10:24:03--  https://download.pytorch.org/models/vgg16-397923af.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.160.200.77, 18.160.200.71, 18.160.200.126, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.160.200.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553433881 (528M) [binary/octet-stream]
Saving to: ‘vgg16-397923af.pth’

vgg16-397923af.pth  100%[===================>] 527.79M   180MB/s    in 2.9s    

2022-06-03 10:24:06 (180 MB/s) - ‘vgg16-397923af.pth’ saved [553433881/553433881]



In [ ]:
#VGG 16 Activations

import torch
import torch.nn as nn
import torch.nn.functional as F


def download_weights(link):
    pass

class VGG_16_ImageNet(nn.Module):
    def __init__(self):
        super(VGG_16_ImageNet, self).__init__()
        self.conv_1_1 = nn.Conv2d(3, 64, 3, 1)
        self.conv_1_2 = nn.Conv2d(64, 64, 3, 1)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.conv_2_1 = nn.Conv2d(64, 128, 3, 1)
        self.conv_2_2 = nn.Conv2d(128, 128, 3, 1)
        self.conv_3_1 = nn.Conv2d(128, 256, 3, 1)
        self.conv_3_2 = nn.Conv2d(256, 256, 3, 1)
        self.conv_3_3 = nn.Conv2d(256, 256, 3, 1)
        self.conv_4_1 = nn.Conv2d(256, 512, 3, 1)
        self.conv_4_2 = nn.Conv2d(512, 512, 3, 1)
        self.conv_4_3 = nn.Conv2d(512, 512, 3, 1)
        self.linear_1 = nn.Linear(25088, 4096)
        self.linear_2 = nn.Linear(4096, 4096)
        self.linear_3 = nn.Linear(4096, 1000)
        self.softmax = torch.nn.Softmax()

    def predict(self, x):
        x = self.softmax(self.forward(x, ppl=False))
        return


    def forward(self, input, ppl=False):
        '''
        Relu
        conv outputs: 
        1_1
        1_2
        3_2
        4_2
        Last linear
        Softmax
        '''
        #x = self.pool(F.relu(self.conv1(x)))
        x_1_1 = F.relu(self.conv_1_1(input)) # 
        x_1_2 = F.relu(self.conv_1_2(x_1_1)) #
        x_1_2_pool = self.max_pool(x_1_2)
        x_2_1 = F.relu(self.conv_2_1(x_1_2_pool))
        x_2_2 = self.max_pool(F.relu(self.conv_2_2(x_2_1)))
        x_3_1 = F.relu(self.conv_3_1(x_2_2))
        x_3_2 = F.relu(self.conv_3_2(x_3_1)) #
        x_3_3 = self.max_pool(F.relu(self.conv_3_3(x_3_2)))
        x_4_1 = F.relu(self.conv_4_1(x_3_3))
        x_4_2 = F.relu(self.conv_4_2(x_4_1)) #
        x_4_3 = self.max_pool(F.relu(self.conv_4_3(x_4_2)))
        x_5_1 = F.relu(self.conv_4_2(x_4_3))
        x_5_2 = F.relu(self.conv_4_2(x_4_2)) #
        x_5_3 = self.max_pool(F.relu(self.conv_4_3(x_5_2)))
        x_linear = torch.flatten(x_5_3, 1)
        x = self.linear_1(x_linear)
        x = self.linear_2(x)
        x = self.linear_3(x)
        
        if ppl is False:
            return x
        else:
            return x_1_1, x_1_2, x_3_2, x_4_2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = VGG_16_ImageNet()
model.load_state_dict(torch.load("/content/vgg16-397923af.pth", map_location=device))
model = torch.load()
x = torch.randn(1, 3, 224, 224)
x_1_1, x_1_2, x_3_2, x_4_2 = model(x, ppl=True)
print(x_1_1.shape)
print(x_1_2.shape)
print(x_3_2.shape)
print(x_4_2.shape)



In [ ]:
from torchvision import models

class VGG16_perceptual(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(VGG16_perceptual, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 4):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 14):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(14, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_1 = h
        h = self.slice2(h)
        h_relu1_2 = h
        h = self.slice3(h)
        h_relu3_2 = h
        h = self.slice4(h)
        h_relu4_2 = h
        return h_relu1_1, h_relu1_2, h_relu3_2, h_relu4_2

model = VGG16_perceptual()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

[Align_Face.py](https://github.com/AmarSaini/Epoching_StyleGan2_Setup/blob/master/align_face.py)

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2022-06-06 13:38:37--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.1’

_predictor_68_face_  12%[=>                  ]   7.85M  2.41MB/s    eta 31s    ^C


In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/shape_predictor_68_face_landmarks.dat.bz2

In [ ]:
"""
brief: face alignment with FFHQ method (https://github.com/NVlabs/ffhq-dataset)
author: lzhbrian (https://lzhbrian.me)
date: 2020.1.5
note: code is heavily borrowed from 
    https://github.com/NVlabs/ffhq-dataset
    http://dlib.net/face_landmark_detection.py.html

requirements:
    apt install cmake
    conda install Pillow numpy scipy
    pip install dlib
    # download face landmark model from: 
    # http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
"""

import numpy as np
import PIL
import PIL.Image
import sys
import os
import glob
import scipy
import scipy.ndimage
import dlib


# download model from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def get_landmark(filepath):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)

    print("Number of faces detected: {}".format(len(dets)))
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.left(), d.top(), d.right(), d.bottom()))
        # Get the landmarks/parts for the face in box d.
        shape = predictor(img, d)
        print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))


    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    # lm is a shape=(68,2) np.array
    return lm

    
def align_face(filepath):
    """
    :param filepath: str
    :return: PIL Image
    """

    lm = get_landmark(filepath)
    
    lm_chin          = lm[0  : 17]  # left-right
    lm_eyebrow_left  = lm[17 : 22]  # left-right
    lm_eyebrow_right = lm[22 : 27]  # left-right
    lm_nose          = lm[27 : 31]  # top-down
    lm_nostrils      = lm[31 : 36]  # top-down
    lm_eye_left      = lm[36 : 42]  # left-clockwise
    lm_eye_right     = lm[42 : 48]  # left-clockwise
    lm_mouth_outer   = lm[48 : 60]  # left-clockwise
    lm_mouth_inner   = lm[60 : 68]  # left-clockwise

    # Calculate auxiliary vectors.
    eye_left     = np.mean(lm_eye_left, axis=0)
    eye_right    = np.mean(lm_eye_right, axis=0)
    eye_avg      = (eye_left + eye_right) * 0.5
    eye_to_eye   = eye_right - eye_left
    mouth_left   = lm_mouth_outer[0]
    mouth_right  = lm_mouth_outer[6]
    mouth_avg    = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg

    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_avg + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2


    # read image
    img = PIL.Image.open(filepath)

    output_size=1024
    transform_size=4096
    enable_padding=True

    # Shrink.
    shrink = int(np.floor(qsize / output_size * 0.5))
    if shrink > 1:
        rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
        img = img.resize(rsize, PIL.Image.ANTIALIAS)
        quad /= shrink
        qsize /= shrink

    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
        img = img.crop(crop)
        quad -= crop[0:2]

    # Pad.
    pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
    if enable_padding and max(pad) > border - 4:
        pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
        img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
        h, w, _ = img.shape
        y, x, _ = np.ogrid[:h, :w, :1]
        mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
        blur = qsize * 0.02
        img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
        img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
        img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
        quad += pad[:2]

    # Transform.
    img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
    if output_size < transform_size:
        img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

    # Save aligned image.
    return img


img =  align_face("/content/bradley_cooper.jpg") # save_img
img = img.save("Brad_Test.png")

Number of faces detected: 1
Detection 0: Left: 118 Top: 333 Right: 504 Bottom: 718
Part 0: (98, 457), Part 1: (104, 507) ...


In [ ]:

img =  align_face("/content/bradley_cooper.jpg") # save_img
img = img.save("Bradley_Test.png")

Number of faces detected: 1
Detection 0: Left: 118 Top: 333 Right: 504 Bottom: 718
Part 0: (98, 457), Part 1: (104, 507) ...


In [ ]:
print(img.size)

(1024, 1024)


In [ ]:
print(type(img))

<class 'PIL.Image.Image'>


In [ ]:
from IPython.display import Image 
display(img)

In [ ]:
import torch
import torch.nn as nn
#from torchvision.io import read_image
import torchvision.transforms as transforms
import PIL


def read_img(image_path):
    '''
    1. Read IMG
    2. Preprocess:
     - cut (eyes are points)
     - resize
    3. Transform to Torch
    '''
    img = PIL.Image.open(image_path)
    transform = transforms.ToTensor() # normalization 0 to 1
    img = transform(img)
    img = torch.unsqueeze(img, 0)
    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    
    return img

def perceptual_loss(model, image_mean, image_embed):
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    features_mean = model(image_mean)
    features_embed = model(image_embed)
    ppl_loss = 0
    for i in range(len(features_mean)):
        ppl_loss += mse_loss(features_mean[i],features_embed[i])
    return ppl_loss

def mse_loss(image_mean, image_embed):
    loss = nn.MSELoss()
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    output = loss(image_mean, image_embed)
    return output

def loss_all(model, image_mean, image_embed, upsample):
    image_mean_up = upsample(image_mean)
    image_embed_up = upsample(image_embed)
    ppl = perceptual_loss(model, image_mean_up, image_embed_up) # tutaj upsample
    mse = mse_loss(image_mean, image_embed)
    return ppl, mse


#img_1 = read_img("/content/Bradley_Test.png")
#img_2 = read_img("/content/Jim_Test.png")


#ppl, mse = loss_all(model, img_1, img_2, upsample)
#print(f"PPL loss: {ppl} \\ MSE loss: {mse}")

In [ ]:

#latents = torch.zeros((1,18,512), requires_grad = True, device = device)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
zero_vector = torch.zeros((1, 18, 512), requires_grad=True, device="cuda") # torch.Size([1, 18, 512])
optim = torch.optim.Adam({zero_vector},lr=0.01,betas=(0.9,0.999),eps=1e-8)
model = VGG16_perceptual().to(device)
zero_vector_2 = torch.zeros((1, 18, 512), requires_grad=True, device="cuda") # torch.Size([1, 18, 512])
def get_embedding(model, G, optimizer, zero_vector, image_to_embed, upsample, step=5000):
    '''
    dopisac .to(device)
    '''
    loss_ = []
    for i in range(step):
        optimizer.zero_grad()
        image_mean = G.synthesis(zero_vector, noise_mode='const', force_fp32=True)
        image_mean = (image_mean+1.0)/2.0 # from union -1|1 to 0/1
        per_loss, mse = loss_all(model, image_mean, image_to_embed, upsample)
        loss = per_loss + mse
        loss.backward()
        optimizer.step()
        loss_.append(loss.detach().cpu().numpy())
    return loss_, zero_vector

In [ ]:
zero_vector_3 = torch.zeros((1, 18, 512), requires_grad=True, device="cuda")
optim = torch.optim.Adam({zero_vector_3},lr=0.01,betas=(0.9,0.999),eps=1e-8)
image_to_embed = read_img("/content/Brad_Test.png")
loss_, zero_vector_3 = get_embedding(model, G, optim, zero_vector_3, image_to_embed, upsample, step=180)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode

In [ ]:
zero_vector_3

tensor([[[-1.8558e-04, -2.5685e-01, -2.0765e-02,  ...,  3.2939e-01,
          -1.1119e-01, -1.5325e-01],
         [ 9.4424e-02, -1.2565e-01,  1.4213e-01,  ...,  7.5901e-02,
           3.0604e-01,  3.1796e-01],
         [-2.7170e-01,  6.5900e-02,  2.8508e-02,  ..., -4.2660e-02,
          -1.4345e-01, -4.1841e-02],
         ...,
         [-3.6786e-02,  1.7231e-01,  2.5252e-01,  ..., -1.2388e-01,
          -1.8544e-02, -1.3794e-01],
         [ 1.6612e-02,  9.8179e-02, -1.5562e-01,  ...,  9.4542e-02,
          -1.1507e-01, -1.0935e-01],
         [ 5.8472e-02, -7.8958e-02, -8.8642e-02,  ..., -1.8686e-01,
           1.7984e-01,  1.1555e-01]]], device='cuda:0', requires_grad=True)

In [ ]:
import numpy as np
import PIL
img_embedded = G.synthesis(zero_vector_3, noise_mode='const', force_fp32=True)
img_embedded = (img_embedded.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img_embedded[0].cpu().numpy(), 'RGB').save(f'Wodeczka180.png')

# TODO:

1. zaczac od random wektor:
wygenerowac 50 wektorow i je usrednic
albo wygenertowac zdjecia dla zero_vectora skoro ma byc srednia

* zerovector - 5k steps i i tak ma wiekszy los ale od tego zaczac


In [ ]:
import torch

def check(image):
    image = torch.unsqueeze(image, 0)
    print(image.shape)
    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    img_p = image.clone()
    img_p = upsample(img_p)
    return img_p

img_p = check(img_1)
img_p.shape

torch.Size([1, 3, 1024, 1024])


torch.Size([1, 3, 256, 256])

In [ ]:
img_1.shape

torch.Size([3, 1024, 1024])

In [ ]:
model = VGG16_perceptual()

In [ ]:
torch.min(img)

tensor(0.)

In [ ]:
# MSE NA 2 ZDJECIACH POPROSTU
# I SPRAWDZIC CZY NA ZNORMALIZOWANYCH SIE CZYMS ROZNI

# Stelgan2 Encoder (Moj pomysl)

[Mapping Network](https://stackoverflow.com/questions/70869211/how-does-mapping-network-in-stylegan-work)
 
 depth 2 --- stylegan 2 ada

In [ ]:
'''
FFHQ from kaggle 512x512
zdjecie 512x512 --> resize --> 256x256 --> dekoder --> (18, 512) --> generator --> Gzdjecie 1024x1024 --> resize --> 256x256
Loss = loss(zdjecie 1024x1024||Gzdjecie 1024x1024)
'''

In [ ]:
# Feature Extractor gotowy np ResNet50

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [2]:
!kaggle datasets download -d arnaud58/flickrfaceshq-dataset-ffhq

100% 19.4G/19.5G [02:37<00:00, 142MB/s]
100% 19.5G/19.5G [02:37<00:00, 133MB/s]


In [3]:
!mkdir "ffhq_dataset"
!mkdir "ffhq_dataset/face"

In [ ]:
!unzip "/content/flickrfaceshq-dataset-ffhq.zip" -d "ffhq_dataset/face"

In [5]:
!rm -r "/content/flickrfaceshq-dataset-ffhq.zip"

In [6]:
!du -sh ffhq_dataset

16G	ffhq_dataset


# Najpierw swoj potem tego chinczyka

1.TODO Przebadac to G co to jest dokladnie (slownik?)

In [ ]:
#FFHQ from kaggle 512x512
#zdjecie 512x512 --> resize --> 256x256 --> dekoder --> (18, 512) --> generator --> Gzdjecie 1024x1024 --> resize --> 256x256
#Loss = loss(zdjecie 1024x1024||Gzdjecie 1024x1024)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models as models 

In [6]:

class StyleganEncoder(nn.Module):
    def __init__(self):
        super(StyleganEncoder, self).__init__()
        self.feature_extractor = self._getResnet()
        self.conv_1 = nn.Conv2d(512, 256, 1)
        self.conv_2 = nn.Conv2d(256, 144, 1)
        self.mapping_network = nn.Sequential(
          nn.Linear(96,96),
          nn.ReLU(),
          nn.Linear(96,96),
          nn.ReLU()
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        batch_dim = x.shape[0]
        x = torch.reshape(x, (batch_dim, 96, 96))
        x = self.mapping_network(x)
        x = torch.reshape(x, (batch_dim, 18, 512))
        return x

    def _getResnet(self):
        resnet = models.resnet18(pretrained=True)
        features = nn.Sequential(*(list(resnet.children())[0:8]))
        for param in features.parameters():
            param.requires_grad = False
        # uciac do basic block 3-8
        # dodac conv2d  output 8|8| 144
        # reshape 96x96
        # mapping network 96x96
        # reshape 18,512
        return features

resnet = StyleganEncoder()
x = torch.randn(10, 3, 256, 256)
output = resnet(x)
output.shape

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

torch.Size([10, 18, 512])

In [7]:
print(resnet)

StyleganEncoder(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

[Tutorial sieci rozlaczanie najlepszy](https://glassboxmedicine.com/2020/12/08/using-predefined-and-pretrained-cnns-in-pytorch-tutorial-with-code/)

In [10]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from torchinfo import summary
summary(resnet, input_size=(1, 3, 256, 256))

Layer (type:depth-idx)                        Output Shape              Param #
StyleganEncoder                               [1, 18, 512]              --
├─Sequential: 1-1                             [1, 512, 8, 8]            --
│    └─Conv2d: 2-1                            [1, 64, 128, 128]         (9,408)
│    └─BatchNorm2d: 2-2                       [1, 64, 128, 128]         (128)
│    └─ReLU: 2-3                              [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                         [1, 64, 64, 64]           --
│    └─Sequential: 2-5                        [1, 64, 64, 64]           --
│    │    └─BasicBlock: 3-1                   [1, 64, 64, 64]           (73,984)
│    │    └─BasicBlock: 3-2                   [1, 64, 64, 64]           (73,984)
│    └─Sequential: 2-6                        [1, 128, 32, 32]          --
│    │    └─BasicBlock: 3-3                   [1, 128, 32, 32]          (230,144)
│    │    └─BasicBlock: 3-4                   [1, 128, 32, 32]      

In [14]:
!gdown https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Downloading...
From: https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
To: /content/ffhq.pkl
100% 382M/382M [00:01<00:00, 209MB/s]


In [16]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 28.70 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [17]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 4.0 MB/s eta 0:01:21tcmalloc: large alloc 1147494400 bytes == 0x39a7c000 @  0x7ff0c1cb9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.3 MB/s eta 0:01:19tcmalloc: large alloc 1434370048 bytes == 0x7e0d2000 @  0x7ff0c1cb9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [1]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 11.3 MB/s 
     |████████████████████████████████| 108 kB 49.0 MB/s 
     |████████████████████████████████| 26.9 MB 2.6 MB/s 


In [2]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [3]:
import torch
import pickle

with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [26]:
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

DATA_PATH = "ffhq_dataset/"
BATCH_SIZE = 16
epochs = 1
encoder = StyleganEncoder().to(device)
data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=TRANSFORM_IMG)
dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-3)

def train_loop(encoder, G, dataloader, epochs):
    loss_tab = []
    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    for i in range(epochs):
        save_loss = 0
        for images, labels in dataloader:
            save_loss +=1
            images = images.to(device)
            latents_vectors = encoder(images)
            generated_images = torch.zeros(BATCH_SIZE, 3, 256, 256).to(device)
            for idx, latent_vector in enumerate(latents_vectors): 
                latent_vector = latent_vector.unsqueeze(0) # dodac batch dim
                img = G.synthesis(latent_vector, noise_mode='const', force_fp32=True)
                img = upsample(img)
                generated_images[idx] = img
            loss = criterion(images, generated_images)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if save_loss == 10:
                loss_tab.append(loss)
                save_loss = 0
    return loss_tab



In [27]:
loss_tab = train_loop(encoder, G, dataloader, epochs)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/funct

OSError: ignored

In [ ]:
loss_tab # 1:30h o 13:30



In [31]:
import PIL
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

img_path = "/content/Brad_Test.png"

img = PIL.Image.open(img_path)


img = TRANSFORM_IMG(img)
img = img.unsqueeze(0).to(device)

latent_img = encoder(img)

In [33]:
import numpy as np
import PIL
img_embedded = G.synthesis(latent_img, noise_mode='const', force_fp32=True)
img_embedded = (img_embedded.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img_embedded[0].cpu().numpy(), 'RGB').save(f'Wodeczka180.png')

In [34]:
torch.save(encoder, 'best-encoder.pt')
torch.save(encoder.state_dict(), 'best-encoder-parameters.pt')

In [35]:
!cp /content/best-encoder-parameters.pt /content/drive/MyDrive/encoder_stylegan2ada
!cp /content/best-encoder.pt /content/drive/MyDrive/encoder_stylegan2ada

# Image to latent

* GPU is mandatory

1. take image
2. preprocess
3. encoder
4. train latnet vector

In [3]:
!cp /content/drive/MyDrive/encoder_stylegan2ada/best-encoder-parameters.pt .

In [4]:
# Git Clone
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [6]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.2 MB/s eta 0:04:35tcmalloc: large alloc 1147494400 bytes == 0x3a726000 @  0x7f253ae93615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████████████▏  | 1055.7 MB 1.1 MB/s eta 0:01:29tcmalloc: large alloc 1434370048 bytes == 0x7ed7c000 @  0x7f253ae93615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [1]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 4.9 MB/s 
     |████████████████████████████████| 108 kB 65.8 MB/s 
     |████████████████████████████████| 26.9 MB 13.1 MB/s 


In [2]:
!gdown https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

Downloading...
From: https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
To: /content/ffhq.pkl
100% 382M/382M [00:01<00:00, 210MB/s]


In [5]:
import sys
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [19]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
import torch
import pickle

with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [7]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/shape_predictor_68_face_landmarks.dat.bz2

--2022-06-08 13:09:19--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  70.3MB/s    in 0.9s    

2022-06-08 13:09:20 (70.3 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [8]:
"""
brief: face alignment with FFHQ method (https://github.com/NVlabs/ffhq-dataset)
author: lzhbrian (https://lzhbrian.me)
date: 2020.1.5
note: code is heavily borrowed from 
    https://github.com/NVlabs/ffhq-dataset
    http://dlib.net/face_landmark_detection.py.html

requirements:
    apt install cmake
    conda install Pillow numpy scipy
    pip install dlib
    # download face landmark model from: 
    # http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
"""

import numpy as np
import PIL
import PIL.Image
import sys
import os
import glob
import scipy
import scipy.ndimage
import dlib


# download model from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
predictor = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

def get_landmark(filepath):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)

    print("Number of faces detected: {}".format(len(dets)))
    for k, d in enumerate(dets):
        print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k, d.left(), d.top(), d.right(), d.bottom()))
        # Get the landmarks/parts for the face in box d.
        shape = predictor(img, d)
        print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))


    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    # lm is a shape=(68,2) np.array
    return lm

    
def align_face(filepath):
    """
    :param filepath: str
    :return: PIL Image
    """

    lm = get_landmark(filepath)
    
    lm_chin          = lm[0  : 17]  # left-right
    lm_eyebrow_left  = lm[17 : 22]  # left-right
    lm_eyebrow_right = lm[22 : 27]  # left-right
    lm_nose          = lm[27 : 31]  # top-down
    lm_nostrils      = lm[31 : 36]  # top-down
    lm_eye_left      = lm[36 : 42]  # left-clockwise
    lm_eye_right     = lm[42 : 48]  # left-clockwise
    lm_mouth_outer   = lm[48 : 60]  # left-clockwise
    lm_mouth_inner   = lm[60 : 68]  # left-clockwise

    # Calculate auxiliary vectors.
    eye_left     = np.mean(lm_eye_left, axis=0)
    eye_right    = np.mean(lm_eye_right, axis=0)
    eye_avg      = (eye_left + eye_right) * 0.5
    eye_to_eye   = eye_right - eye_left
    mouth_left   = lm_mouth_outer[0]
    mouth_right  = lm_mouth_outer[6]
    mouth_avg    = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg

    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_avg + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2


    # read image
    img = PIL.Image.open(filepath)

    output_size=1024
    transform_size=4096
    enable_padding=True

    # Shrink.
    shrink = int(np.floor(qsize / output_size * 0.5))
    if shrink > 1:
        rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
        img = img.resize(rsize, PIL.Image.ANTIALIAS)
        quad /= shrink
        qsize /= shrink

    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
        img = img.crop(crop)
        quad -= crop[0:2]

    # Pad.
    pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
    if enable_padding and max(pad) > border - 4:
        pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
        img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
        h, w, _ = img.shape
        y, x, _ = np.ogrid[:h, :w, :1]
        mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
        blur = qsize * 0.02
        img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
        img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
        img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
        quad += pad[:2]

    # Transform.
    img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
    if output_size < transform_size:
        img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

    # Save aligned image.
    return img




In [14]:
import torch.nn as nn
from torchvision import models as models
import torch.nn.functional as F

class StyleganEncoder(nn.Module):
    def __init__(self):
        super(StyleganEncoder, self).__init__()
        self.feature_extractor = self._getResnet()
        self.conv_1 = nn.Conv2d(512, 256, 1)
        self.conv_2 = nn.Conv2d(256, 144, 1)
        self.mapping_network = nn.Sequential(
          nn.Linear(96,96),
          nn.ReLU(),
          nn.Linear(96,96),
          nn.ReLU()
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        batch_dim = x.shape[0]
        x = torch.reshape(x, (batch_dim, 96, 96))
        x = self.mapping_network(x)
        x = torch.reshape(x, (batch_dim, 18, 512))
        return x

    def _getResnet(self):
        resnet = models.resnet18(pretrained=True)
        features = nn.Sequential(*(list(resnet.children())[0:8]))
        for param in features.parameters():
            param.requires_grad = False
        # uciac do basic block 3-8
        # dodac conv2d  output 8|8| 144
        # reshape 96x96
        # mapping network 96x96
        # reshape 18,512
        return features

resnet = StyleganEncoder()
x = torch.randn(10, 3, 256, 256)
output = resnet(x)
output.shape

torch.Size([10, 18, 512])

In [22]:
import PIL
import numpy as np
import torchvision.transforms as transforms

weights_pth = "/content/best-encoder-parameters.pt"
img_path = "/content/bradley_cooper.jpg"

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

img =  align_face(img_path) # save_img
img = img.save("face_aligned.png")

def preprocess_image(img_path):
    img = PIL.Image.open(img_path)
    img = TRANSFORM_IMG(img)
    img = img.unsqueeze(0).to(device)
    return img

def load_encoder(weights_pth):
    encoder = StyleganEncoder().to(device)
    encoder.load_state_dict(torch.load(weights_pth, map_location=device))
    return encoder

def get_encoder_latent(img, save=False):
    latent_img = encoder(img)
    if save is True:
        img_embedded = G.synthesis(latent_img, noise_mode='const', force_fp32=True)
        img_embedded = (img_embedded.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img_embedded[0].cpu().numpy(), 'RGB').save(f'encoder_img.png')
    return latent_img


encoder = load_encoder(weights_pth)
img_to_encoder = preprocess_image("face_aligned.png")
latent_encoder = get_encoder_latent(img_to_encoder, save=True)

Number of faces detected: 1
Detection 0: Left: 118 Top: 333 Right: 504 Bottom: 718
Part 0: (98, 457), Part 1: (104, 507) ...


In [23]:
import torch
import torch.nn as nn
#from torchvision.io import read_image
import torchvision.transforms as transforms
import PIL


def read_img(image_path):
    '''
    1. Read IMG
    2. Preprocess:
     - cut (eyes are points)
     - resize
    3. Transform to Torch
    '''
    img = PIL.Image.open(image_path)
    transform = transforms.ToTensor() # normalization 0 to 1
    img = transform(img)
    img = torch.unsqueeze(img, 0)
    upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
    
    return img

def perceptual_loss(model, image_mean, image_embed):
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    features_mean = model(image_mean)
    features_embed = model(image_embed)
    ppl_loss = 0
    for i in range(len(features_mean)):
        ppl_loss += mse_loss(features_mean[i],features_embed[i])
    return ppl_loss

def mse_loss(image_mean, image_embed):
    loss = nn.MSELoss()
    image_mean = image_mean.to(device)
    image_embed = image_embed.to(device)
    output = loss(image_mean, image_embed)
    return output

def loss_all(model, image_mean, image_embed, upsample):
    image_mean_up = upsample(image_mean)
    image_embed_up = upsample(image_embed)
    ppl = perceptual_loss(model, image_mean_up, image_embed_up) # tutaj upsample
    mse = mse_loss(image_mean, image_embed)
    return ppl, mse


#img_1 = read_img("/content/Bradley_Test.png")
#img_2 = read_img("/content/Jim_Test.png")


#ppl, mse = loss_all(model, img_1, img_2, upsample)
#print(f"PPL loss: {ppl} \\ MSE loss: {mse}")

In [24]:
from torchvision import models

class VGG16_perceptual(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(VGG16_perceptual, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 4):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 14):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(14, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_1 = h
        h = self.slice2(h)
        h_relu1_2 = h
        h = self.slice3(h)
        h_relu3_2 = h
        h = self.slice4(h)
        h_relu4_2 = h
        return h_relu1_1, h_relu1_2, h_relu3_2, h_relu4_2

In [26]:
upsample = torch.nn.Upsample(scale_factor = 256/1024, mode = 'bilinear')
latent_encoder = torch.tensor(latent_encoder, requires_grad=True)
optim = torch.optim.Adam({latent_encoder},lr=0.01,betas=(0.9,0.999),eps=1e-8)
model = VGG16_perceptual().to(device)

def get_embedding(model, G, optimizer, latent_encoder, image_to_embed, upsample, step=5000):
    '''
    dopisac .to(device)
    '''
    loss_ = []
    for i in range(step):
        optimizer.zero_grad()
        image_mean = G.synthesis(latent_encoder, noise_mode='const', force_fp32=True)
        image_mean = (image_mean+1.0)/2.0 # from union -1|1 to 0/1
        per_loss, mse = loss_all(model, image_mean, image_to_embed, upsample)
        loss = per_loss + mse
        loss.backward()
        optimizer.step()
        loss_.append(loss.detach().cpu().numpy())
    return loss_, latent_encoder

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [27]:
image_to_embed = read_img("face_aligned.png")
loss_, latent_projector = get_embedding(model, G, optim, latent_encoder, image_to_embed, upsample, step=300)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode

In [28]:
img_embedded = G.synthesis(latent_projector, noise_mode='const', force_fp32=True)
img_embedded = (img_embedded.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
PIL.Image.fromarray(img_embedded[0].cpu().numpy(), 'RGB').save(f'Projector.png')